# Birdsongs - 06.- Preprocesando Datos - Convirtiendo Audio a Datos


Convierte los ficheros de audio de un repositorio local a ficheros de datos en formato numpy array. En caso de interrupción se puede volver a lanzar y continua con el proceso copiando los restantes (si se quiere empezar de nuevo hay que eliminar los directorios de destino).

La estructura de directorios de destino es la misma que los ficheros de audio. Esto permite luego tratar con los datos de forma más eficiente que tener que estar transformando los datos "al vuelo".

![directorio](./resources/directorioaudios.png)


Va procesando los audios que existen en cada subdirectorio y va creando un directorio de datos, con los ficheros numpy contiendo la información de la grabación. Vamos a tratar con dos tipos de datos

### a) [Mel scale Spectrogram](https://en.wikipedia.org/wiki/Mel_scale)

>The mel scale, named by Stevens, Volkmann, and Newman in 1937,[1] is a perceptual scale of pitches judged by listeners to be equal in distance from one another. The reference point between this scale and normal frequency measurement is defined by assigning a perceptual pitch of 1000 mels to a 1000 Hz tone, 40 dB above the listener's threshold. Above about 500 Hz, increasingly large intervals are judged by listeners to produce equal pitch increments. As a result, four octaves on the hertz scale above 500 Hz are judged to comprise about two octaves on the mel scale. The name mel comes from the word melody to indicate that the scale is based on pitch comparisons. 



MEL spectrogram es bastante utilizado en procesos de machine learning para tratamiento de audio y voz. 


https://github.com/adanRivas/CNN-Audio-Classifier-with-Keras-Tensorflow/blob/master/save_melspectograms.ipynb



### b) [Welch's method Periodogram](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.welch.html#scipy.signal.welch)

>Welch’s method computes an estimate of the power spectral density by dividing the data into overlapping segments, computing a modified periodogram for each segment and averaging the periodograms.
>
>In physics, engineering, and applied mathematics, Welch's method, named after P.D. Welch, is used for estimating the power of a signal at different frequencies: that is, it is an approach to spectral density estimation. The method is based on the concept of using periodogram spectrum estimates, which are the result of converting a signal from the time domain to the frequency domain. Welch's method is an improvement on the standard periodogram spectrum estimating method and on Bartlett's method, in that it reduces noise in the estimated power spectra in exchange for reducing the frequency resolution. Due to the noise caused by imperfect and finite data, the noise reduction from Welch's method is often desired. 


El resultado de este notebook es la generación de un directorio con los audios transformados a datos.


## 1.- Librerías

Cargamos las librerías de uso común.

Para el tratamiento de los audios vamos a utilizar [**libROsa**](http://librosa.github.io/librosa/). Es una librería que nos permite convertir los audios en datos, y a su vez, transformarlos en información útil en el campo de análisis de señales: oscilogramas, espectrogramas, MFCC,... Además, la ventaja que tiene frente a otras librerías, es que permite leer los audios directamente en mp3, cosa que otras librerías del tipo [PySoundfile](https://pypi.org/project/SoundFile/) o  [Soundfile](https://pysoundfile.readthedocs.io/en/0.9.0/) no soportan.

> LibROSA is a python package for music and audio analysis. It provides the building blocks necessary to create music information retrieval systems.

      conda install -c conda-forge librosa 
     
Y también utilizaremos [**scipy**](https://docs.scipy.org/doc/scipy/reference/), para el cálculo de periodogramas.

>SciPy (pronounced “Sigh Pie”) is a Python-based ecosystem of open-source software for mathematics, science, and engineering. In particular, these are some of the core packages:
     
      conda install -c anaconda scipy
      



In [ ]:
# importar librerías
import os
import numpy as np
import librosa
import librosa.display
import scipy.signal as signal


## 2.- Funciones

Funciones utilizadas en el notebook

### Recupera directorios

Los ficheros de audio se encuentran localizados en un directorio raíz, y dentro del correspondiente subdirectorio al que pertenece la especie. Realizando un dir de los directorios contenidos en el directorio raíz, tendremos una lista con todas las especies que forman parte del dataset.

In [ ]:
#----------------------------------------------------------------------------
# get_specie_names(path)
#  argumentos: 
#      path: directorio de audios
#----------------------------------------------------------------------------
def get_specie_names(path):  
    specie_names = os.listdir(path)
    return specie_names

### Procesa audios

Procesa los audios que existen en cada subdirectorio y va creando un directorio de datos, con los ficheros numpy contiendo la información de la grabación en el formato pasado por parámetro.

##### No se generan MFCC, para ello, descomentar el código


In [ ]:
#----------------------------------------------------------------------------
# process_audio(audiopath, datatype, melpath, mfccpath, welchpath)
#  argumentos: 
#      audiopath: repositorio de audios
#      datatype: tipo de datos
#      melpath: repositorio de destino para Espectrogramas de MEL
#      mfccpath: repositorio de destino para MFCC
#      welchpath: repositorio de destino para periodogramas Welch
#----------------------------------------------------------------------------
def process_audio(audiopath, datatype, melpath, mfccpath, welchpath):
    print(">>>> procesando audios...")
    
    # crea el directorio raiz de datos
    if datatype == CT_MEL:
        if not os.path.exists(melpath):
            os.mkdir(melpath)

        #if not os.path.exists(mfccpath):
        #    os.mkdir(mfccpath)
            
    elif datatype == CT_WELCH:
        if not os.path.exists(welchpath):
            os.mkdir(welchpath)
        
    # recupera la lista de especies (directorios) a tratar
    specie_names = get_specie_names(audiopath)
    number_species = len(specie_names)
   
    # itera sobre cada directorio y va generando un fichero con los MEL spectrogram
    for idx, specie_name in enumerate(specie_names):
        # crea directorio destino de la especie según el tipo
        if datatype == CT_MEL:
            meldir = os.path.join(melpath, specie_name)
            if not os.path.exists(meldir):
                os.mkdir(meldir)

           # mfccdir = os.path.join(mfccpath, specie_name)
           # if not os.path.exists(mfccdir):
           #     os.mkdir(mfccdir)

        elif datatype == CT_WELCH:
            welchdir = os.path.join(welchpath, specie_name)
            if not os.path.exists(welchdir):
                os.mkdir(welchdir)
                
                
        # recupera los ficheros de audio existentes para esta especie
        specie_dir = os.path.join(audiopath, specie_name)
        specie_files = os.listdir(specie_dir)
        number_files = len(specie_files)
        number_load = number_files
        
        print(' Specie name = {:14s} - {:3d}'.format(specie_name,idx),
               ", ",number_files," files in this specie", sep=" ", end='\r', flush=True)

        # itera sobre la lista de audios recuperada
        printevery = 20
        
        for idx2, infilename in enumerate(specie_files):
            # fichero audio origen
            file_path = specie_dir + '/' + infilename

            # muestra avance
            if (0 == idx2 % printevery):
                print('\r Loading specie: {:14s} ({:2d} of {:2d} species)'.format(specie_name,idx+1,number_species),
                       ", file ",idx2+1," of ",number_load,": ",file_path, sep=" ", end='\r', flush=True)

            # nombre del audio
            file_name = infilename[:infilename.find('.')]
            
            # MEL espectrograma
            if datatype == CT_MEL:
            
                # si no se ha generado previamente, procedemos a generarlo    
                melfile = meldir + '/' + file_name +'_mel.npy'
                #mfccfile = mfccdir + '/' + file_name +'_mfcc.npy'
            
                if not os.path.isfile(melfile):
                    # lee fichero
                    aud, sr = librosa.load(file_path, sr=None)

                    # genera el log MEL spectrogram
                    melgram = librosa.feature.melspectrogram(aud, sr=sr, n_mels=128)
                    log_melgram = librosa.amplitude_to_db(melgram, ref=np.max)

                    # salva fichero numpy 
                    np.save(melfile,log_melgram)
                    
                    # genera el MFCC
                    #mfcc = librosa.feature.mfcc(S=log_melgram, n_mfcc=20)
                    #np.save(mfccfile,mfcc)
                    
            # Welch Periodogram
            elif datatype == CT_WELCH:
                # si no se ha generado previamente, procedemos a generarlo    
                welchfile = welchdir + '/' + file_name +'_welch.npy'
            
                if not os.path.isfile(welchfile):
                    # lee fichero
                    aud, sr = librosa.load(file_path, sr=None)

                    # genera el periodograma Welch
                    welchgram = signal.welch(aud, sr, "flattop", 11024, scaling="spectrum")

                    # salva fichero numpy 
                    np.save(welchfile,welchgram)


## 3.- Convierte audio en datos

Lanza el proceso de conversión de todos los audios localizados en el repositorio al formato de dato especificado

In [ ]:
%%time

# constantes
CT_MEL = 'MEL'
CT_WELCH = 'WELCH'

# repositorio con los audios
audiopath = './audio'

# tipo de dato a generar
datatype = CT_WELCH

# repositorios de generación de los datos
melpath= './data/mel2'
mfccpath= './data/mfcc2'
welchpath= './data/welch2'

# genera datos a partir de los audios
process_audio(audiopath, datatype, melpath, mfccpath, welchpath)
